### import

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
import datetime

### function

In [2]:
주민조건 = [2,3,4,5,6,7,8,9,2,3,4,5]
법인조건 = [1,2,1,2,1,2,1,2,1,2,1,2]
사업자조건 = [1,3,7,1,3,7,1,3,5]
# 주민등록번호 유효성 검사 > 통과시 개인. 개인인데 주민번호 오류인것, 법인등록번호가 유효성 통과한 경우 확인 필요
############################# 사업자 검사 추가, 14자리이나 000으로 시작하는 경우, 000 제거하고 사업자 등록번호 검사,통과하면 법인, 통과하지 못하면 개인.
############################# 셋다 통과하지 못하거나, 10자리인데 사업자 번호 통과하지 못하면 유효하지 않은 번호로 따로 열 만들어서 적어주자. 
def only_num(regit_num:str) :
    return re.sub("[^0-9]", "", regit_num)

def 식별번호유효성검사 (regit_num:str, cond_list:list, minus_num:int, mod_num:int) :
    sum = 0
    for i in range(len(regit_num)-1) :
        sum += int(regit_num[i]) * cond_list[i]
    if len(cond_list) == 9 : # 사업자번호인경우
        sum += int(regit_num[8]) * cond_list[8] // 10
    return int(regit_num[-1]) == (minus_num - (sum % mod_num)) % 10 # 일의자리만 얻기 위해 % 10

def 주민번호유효성검사(regit_num:str) :
    regit_num = only_num(regit_num)
    if len(regit_num) == 13 :
        month = int(regit_num[2:4])
        day = int(regit_num[4:6])

        # 월 일 유효성 검증
        if month > 12 or day > 31 : return False
        elif month in [4,6,9,11] : 
            if day > 30 : return False
        elif month == 2 :
            if day > 29 : return False

        # 주민번호 유효성 검증
        # 내국인
        number = 0
    
        # 내국인
        if regit_num[6] in ["1","2","3","4","9","0"] : 
            return 식별번호유효성검사(regit_num, 주민조건, minus_num=11, mod_num=11)
        # 외국인 
        else : 
            return 식별번호유효성검사(regit_num, 주민조건, minus_num=13, mod_num=11)
    else :
        return False

# 법인등록번호 유효성 체크, 인터넷등기소 > 법입 > 열람
def 법인번호유효성검사(regit_num:str) : # 0으로시작하는 관서번호는 없음
    regit_num = only_num(regit_num)
    if len(regit_num) != 13 : return False
    else :
        # 등기관서번호 # 첫줄은 옛날 관서번호 : 추가중 # 옛날번호는 유효성검사 실패할 수 있음.
        등기관서번호 = (1154,1156,1159,1115,1741,1742,1753,1761,1762,1763,1842,1850,1901,2041,2042,2048,2063,2150,2151,2342,1301,
        1101,1103,1201,1211,1244,1245,1311,1312,1313,1314,1341,1342,1343,1344,1345,1346,1348,1349,1354,1355,
        1356,1357,1358,1359,1401,1411,1412,1413,1414,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1501,1511,1512,
        1513,1541,1542,1543,1544,1545,1546,1601,1611,1612,1613,1614,1615,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,
        1652,1701,1711,1712,1713,1714,1715,1716,1717,1718,1743,1744,1745,1747,1748,1749,1750,1751,1752,1754,1755,1756,1757,1758,
        1759,1760,1801,1811,1841,1843,1844,1847,1849,1851,1911,1912,1913,1914,1915,1941,1942,1943,1944,1945,1946,1947,1948,1949,
        1950,1951,1952,1953,1954,1955,2001,2011,2012,2013,2014,2043,2044,2045,2046,2047,2049,2050,2052,2053,2054,2055,2056,2057,
        2058,2059,2060,2061,2062,2101,2111,2112,2113,2141,2142,2143,2144,2145,2146,2147,2148,2149,2201,2241,2301,2341,2401,2501,
        2601,2701,2802,2841,2842,2843,2844,2845,2846,2847,2849,2850)                                                
        법인종류번호 = (11,12,13,14,15,21,22,31,32,33,34,35,36,38,39,40,41,43,44,46,47,48,49,51,52,53,54,55,61,71,81,82,83,84,85,86,42,45,50) # 42,45,50 는 옛날 법인종류
        if int(regit_num[:4]) not in 등기관서번호 : 
            return False
        # 법인종류번호
        elif int(regit_num[4:6]) not in 법인종류번호 : 
            return False
        else :
            # 유효성 검증
            return 식별번호유효성검사(regit_num, 법인조건, minus_num=10, mod_num=10)
            
# 홈텍스 > 조회/발급 > 사업자상태 > 등록번호/주민번호 조회 or bizno.net
def 사업자번호유효성검사 (regit_num : str) :
    regit_num = only_num(regit_num)
    if len(regit_num) != 10 : return False
    else :
        # 유효성 검사
        return 식별번호유효성검사(regit_num, 사업자조건, minus_num=10, mod_num=10)

# 주민번호로 성별 체크
def 성별분류(regit_num:str, sex:str) :
    regit_num = only_num(regit_num)
    # 개인
    if sex == "개인" :
        # 홀수면 남자
        if int(regit_num[6]) % 2 == 1 : return "남자"
        else : return "여자"
    # 법인 또는 오류
    else : return sex

In [6]:
주민번호유효성검사("6408251449216")

True

### 파일 읽기

In [6]:
###########################################
wd = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인법인,성별\솔림"
file_name = "성별_보증인"
ext = ".xlsx"
########################################### ######
file = file_name + ext
if ext == "pkl" : 
    df_ori = pd.read_pickle(join(wd, file))
else :
    df_ori = pd.read_excel(join(wd,file), dtype="str").fillna("")

# 채무자조회 : 매각, 환매, 종결 건제외 (보증인은 불러와야함)
if re.search("채무자", file_name) : 
    out = re.compile("매각|환매|종결")
    df = df_ori[~df_ori.채무상태.str.contains(out) & ~df_ori.담당자.str.contains(out)].copy().reset_index(drop=True)
elif re.search("보증인", file_name) : 
    out = re.compile("매각|환매|종결|면책|면탈|사망|시효|포기|완제")
    df = df_ori[~df_ori.보증인상태.str.contains(out) & ~df_ori.담당자.str.contains(out)].copy().reset_index(drop=True)
    # 컬럼명 변경
    df.rename(columns={"보증인성명인":"성명"}, inplace=True)

c:\Users\sl\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### 개인법인 체크

In [14]:
# 주민번호 유효성 검사 결과. # 채무자, 보증인 모두 "주민번호인"
# 주민번호유효성결과 열 추가
result1 = pd.Series(df.주민번호인.apply(주민번호유효성검사))
df = pd.concat([df, result1.rename("주민번호유효성결과")], axis=1)

result2 = df.주민번호인.apply(법인번호유효성검사)
df = pd.concat([df, result2.rename("법인번호유효성결과")], axis=1)

result3 = df.주민번호인.apply(사업자번호유효성검사)
df = pd.concat([df, result3.rename("사업자번호유효성결과")], axis=1)

In [15]:
# 새로운 개인법인 작성
x = df["주민번호유효성결과"]
y = df["법인번호유효성결과"]  # 연일기계(115411-0005167) 법인번호 맞지만 유효성 오류남. x,y모두 참인 경우 더러 있음
z = df["사업자번호유효성결과"]
# 개인, 법인, 오류 여부를 나타내는 조건 평가
cond1 = x & ~y & ~z
cond2 = (~x & y & ~z) | (~x & ~y & z)

# 조건을 기반으로 값을 설정
new_values = np.select([cond1, cond2], ["개인", "법인"], default="오류")
df = pd.concat([df, pd.DataFrame({"new개인법인": new_values})], axis=1)

In [16]:
# 유효성 검사 결과가 기존 전산값과 다른 것만 엑셀파일로
# 채무자
if re.search("채무자", file_name) : 
    cols_for_save = ["채무자키", "성명", "주민번호인", "개인법인", "new개인법인", "주민번호유효성결과", "법인번호유효성결과", "사업자번호유효성결과"]
    df[df["new개인법인"] != df["개인법인"]][cols_for_save].to_excel(join(wd,"check_개인법인_"+file_name+".xlsx"), index=False)
# 보증인 : 개인법인으로 성별과 new개인법인에서 도출한 new성별 비교
# 엑셀다운자료에는 개인법인 없지만, 업로드 및 수정에는 있음
else :
    df["new성별"] = df.apply(lambda row: 성별분류(row["주민번호인"], row["new개인법인"]), axis=1)
    cols_for_save = ["채무자키", "보증인키", "보증인성명인", "주민번호인", "성별", "new성별", "new개인법인", "주민번호유효성결과", "법인번호유효성결과", "사업자번호유효성결과"]
    # 성별 다른 경우만 저장
    df[df["new성별"] != df["성별"]][cols_for_save].to_excel(join(wd,"check_개인법인성별"+file_name+".xlsx"), index=False)

### 개별식별번호 유효성 체크

In [30]:
# 확인후 최종점검
wd = r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인법인,성별"
temp = pd.read_excel(join(wd, "check개인법인~.xlsx"), dtype=str)[["채무자키","주민번호인","사업자번호인"]]

In [52]:
temp[temp.사업자번호인.notna()]

,채무자키,주민번호인,사업자번호인
0,20417801,214914-0011193,403-81-28448
1,20413623,214714-0000546,405-81-03934
11,20412836,110111-0299910,120-06-70160
14,20429478,110111-0798102,105-81-71551
19,20413902,110111-1939680,211-83-95867
25,20429662,NaN,112-81-49080
26,20429668,111511-0000566,127-81-00677
27,20418301,115411-0005167,127-81-19268
28,20429663,NaN,128-81-32959
29,20429664,NaN,204-81-62500


In [54]:
temp.loc[temp.사업자번호인.notna(),"사업자번호인"].apply(사업자번호유효성검사)

0      True
1      True
11     True
14     True
19    False
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
36     True
66     True
Name: 사업자번호인, dtype: bool

In [5]:
regit_num = "620118-2447021"
print(f"주민 : {주민번호유효성검사(regit_num)}, 법인 : {법인번호유효성검사(regit_num)}, 사업자 : {사업자번호유효성검사(regit_num)}")

주민 : True, 법인 : False, 사업자 : False


### 성별 체크 - 차주만 
- 보증인은 개인법인열이 조회파일에는 없어서 한꺼번에 함(테이블 항목에는 있음)
- 따라서 업데이트 했으면 동일 로직이기 때문에 결과가 다를 수 없음

In [8]:
# 개인법인 이상없는지 확인이 끝나면 작업
if re.search("채무자", file_name) : 
    df["new성별"] = df.apply(lambda row: 성별분류(row["주민번호인"], row["개인법인"]), axis=1)
    cols_for_save = ["채무자키", "성명", "주민번호인", "담당자", "채무상태", "성별", "new성별"]
else : 
    df["new성별"] = df.apply(lambda row: 성별분류(row["주민번호인"], row["성별"]), axis=1)
    cols_for_save = ["채무자키","보증인키", "성명", "주민번호인", "담당자", "성별", "new성별"]
# 성별 다른 경우만 저장
df[df["new성별"] != df["성별"]][cols_for_save].to_excel(join(wd,"check_성별_"+file_name+".xlsx"), index=False)

### 국세청 사업자 휴폐업 조회

In [12]:
import requests

HOMETAX_URL = 'https://teht.hometax.go.kr/wqAction.do?actionId=ATTABZAA001R08&screenId=UTEABAAA13&popupYn=false&realScreenId='

def checkTaxStatus (사업자등록번호):

	#remove '-' from bizId
	removed = bizId.replace ('-','')

	#make XML
	xml = '<map id=""><pubcUserNo/><mobYn>N</mobYn><inqrTrgtClCd>1</inqrTrgtClCd><txprDscmNo>'\
	+ removed +'</txprDscmNo><dongCode></dongCode><psbSearch></psbSearch><map id="userReqInfoVO"/></map>'
	headers = {'Content-Type': 'application/xml'}
	
	r = requests.post (HOMETAX_URL, data=xml, headers=headers).text

	begin_tag = '<trtCntn>'
	end_tag = '</trtCntn>'
	begin_idx = r.find (begin_tag)
	end_idx = r.find(end_tag)

	result = r[begin_idx+9: end_idx]
	print (result)

	return result

In [13]:
사업자등록번호 = "308-81-04817"
checkTaxStatus(사업자등록번호)

폐업자 (부가가치세 일반과세자, 폐업일자:1998-10-20) 입니다.


'폐업자 (부가가치세 일반과세자, 폐업일자:1998-10-20) 입니다.'

### 통신사업판매자(대부분의 사업체?) 조회 - 사업자등록번호/업체명

In [35]:
# 통합본 파일 읽기 (모든 칼럼 str)
df = pd.read_pickle(r"D:\3.자산\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\check\개인법인,성별\통신판매사업자_공정거래위원회_230810\통신판매사업자_ALL_전지역통합_230810.pkl")
df.head(1)

,통신판매번호,신고기관명,상호,사업자등록번호,법인여부,대표자명,전화번호,전자우편,신고일자,사업장소재지,사업장소재지(도로명),업소상태,신고기관 대표연락처,판매방식,취급품목,인터넷도메인,호스트서버소재지
0,2004-01914,서울특별시 종로구,지성의료기,,개인,김정구,02 2213 1367,webmast**@chimdo/com,,서울특별시 종로구 종로**가********,서울특별시 종로구 대학로**길 ** (종*****,통신판매업 신고,02-2148-2294,,,www.chimdo.com,양천구신정동973-35


In [36]:
# 상호검색
df[df["상호"].str.contains("수한금속")]
# 결측치(float) 있을 때
# df[df["상호"].notnull() &  df["상호"].str.contains("레미콘")]

,통신판매번호,신고기관명,상호,사업자등록번호,법인여부,대표자명,전화번호,전자우편,신고일자,사업장소재지,사업장소재지(도로명),업소상태,신고기관 대표연락처,판매방식,취급품목,인터넷도메인,호스트서버소재지
